# Setup
1. install python (3.8) and git
2. install virtualenv e.g. with `<path to python> -m to pip install virtualenv`
3. create a virtual environment that uses python 3.8 using virtualenv e.g. with `<path to python> -m virtualenv "<path to virtual environment>"`
4. activate the virtual environment e.g. on windows with `.\<pathto env>\Scripts\activate`
5. install vscode and python extension
6. git clone brainlit (https://github.com/neurodata/brainlit) and neuroglancer (https://github.com/google/neuroglancer)
7. install brainlit from source e.g. `pip install -e .` in brainlit folder 
8. Place `test-czi.zip` file in `brainlit/experiments/sriramg/data/` and unzip it

In [ ]:
from brainlit.utils.write import czi_to_zarr, zarr_to_omezarr
import zarr
from cloudvolume import CloudVolume
import json
from pathlib import Path

# Convert czi to zarr

In [ ]:
project_path = "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi"  # "E:\\Projects\\KolodkinLab\\Sriram\\brainlit-tracing\\brainlit\\experiments\\sriram\\data\\" "C:\\Users\\Sriram Sudarsanam\\Desktop\\NeuroglancerTrial\\"

project_path = Path(project_path)
czi_path = project_path / "test.czi"  # path to czi image
out_dir = (
    project_path  # path to directory where zarr should be made, should end in slash
)

In [ ]:
zarr_paths = czi_to_zarr(czi_path=czi_path, out_dir=out_dir, fg_channel=1, parallel=1)

# Convert zarr to ome-zarr

In [ ]:
out_path = project_path / "fg_ome.zarr"  # path of ome zarr to be made

In [ ]:
zarr_to_omezarr(zarr_path=zarr_paths[0], out_path=out_path)

# View ome-zarr on neuroglancer

1. Use https://github.com/google/neuroglancer/blob/master/cors_webserver.py to serve the data e.g. `python cors_webserver.py -d "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data" -p 9010`
2. Launch neuroglancer app e.g. https://neuroglancer-demo.appspot.com
3. Add layer to neuroglancer app e.g. zarr://http://127.0.0.1:9010/sriram-adipo-brain1-im3/fg_ome.zarr

# Add trace layer

In [ ]:
traces_dir = out_dir / "traces"
z = zarr.open_array(out_dir / "fg_ome.zarr" / "0")
volume_size = z.shape
chunk_size = z.chunks
resolution = [1000000000, 1000000000, 1000000000]

outpath = f"precomputed://file://" + str(traces_dir)

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16",
    encoding="raw",
    resolution=resolution,  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=chunk_size,  # units are voxels
    volume_size=volume_size,  # e.g. a cubic millimeter dataset
    skeletons="skeletons",
)
vol = CloudVolume(outpath, info=info, compress=False)
vol.commit_info()
vol.skeleton.meta.commit_info()

# remove vertex type attribute because it is a uint8 and incompatible with neuroglancer
info_path = traces_dir / "skeletons/info"
with open(info_path) as f:
    data = json.load(f)
    for i, attr in enumerate(data["vertex_attributes"]):
        if attr["id"] == "vertex_types":
            data["vertex_attributes"].pop(i)
            break

with open(info_path, "w") as f:
    json.dump(data, f)

# Ready to trace
Use `trace.py` to trace neurons

# Optional: Activate ViterBrain

You will need to download ilastik: https://www.ilastik.org/download.html

Update `ilastik_program_path` below, e.g. for Mac/Linux use the path to run_ilastik.sh, for Windows, use the path to ilastik.exe

In [ ]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

In [ ]:
ilastik_program_path = "/Applications/ilastik-1.4.0b21-OSX.app/Contents/ilastik-release/run_ilastik.sh"  # path to ilastik executable e.g. for windows something like "\Program Files\ilastik-1.3.2\ilastik.exe"


image_path = out_dir / "fg_ome.zarr" / "0"
z_im = zarr.open_array(image_path)
ilastik_project_path = out_dir / "ex_fg.ilp"
chunk_size = [c * 2 for c in z_im.chunks]
data_bin = out_dir / "data_bin/"

prob_path = out_dir / "probs.zarr"
fragment_path = out_dir / "labels.zarr"
tiered_path = out_dir / "tiered.zarr"
states_path = out_dir / "nx.pickle"

sg = state_generation(
    image_path=image_path,
    new_layers_dir=out_dir,
    ilastik_program_path=ilastik_program_path,
    ilastik_project_path=ilastik_project_path,
    chunk_size=chunk_size,
    parallel=6,
    #   prob_path = prob_path,
    #   fragment_path=fragment_path,
    #   tiered_path=tiered_path,
    #   states_path = states_path
)

In [ ]:
sg.predict(data_bin=data_bin)
sg.compute_frags()
sg.compute_soma_lbls()
sg.compute_image_tiered()
sg.compute_states()
sg.compute_edge_weights()

# Other

In [ ]:
from skimage import io
import numpy as np

z_im = zarr.open(image_path)
subvol = z[:40, :40, :40]
io.imsave(out_dir + "/train.tif", subvol)

In [ ]:
import pickle
from brainlit.algorithms.connect_fragments.viterbrain import explain_viterbrain

with open(
    "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/viterbrain.pickle",
    "rb",
) as handle:
    vb = pickle.load(handle)

In [ ]:
explain_viterbrain(vb, [558, 549, 4], [480, 517, 2], [])

vs fragments [1176, 1173, 1172]